你刚看到这些数据可能会比较懵，因为这些数据缺少标注。我们在收集整理数据的时候，一定要对数据做标注，数据表头很重要。
<img src='./images/11-01.png'>
* 这是一家服装店统计的会员数据。
* 最上面的一行是列坐标，最左侧一列是行坐标。
* 列坐标中，第 0 列代表的是序号，第 1 列代表的会员的姓名，第 2 列代表年龄，第 3 列代表体重，第 4 ~ 6 列代表男性会员的三围尺寸，第 7~9 列代表女性会员的三围尺寸。

#### 好的数据分析师必定是一名数据清洗高手，要知道在整个数据分析过程中，不论是在时间还是功夫上，数据清洗大概都占到了 80%。

### 数据质量的准则
在这里，我将数据清洗规则总结为以下 4 个关键点，统一起来叫“完全合一”。按照以下的原则，我们能解决数据清理中遇到的大部分问题，使得数据<b>标准、干净、连续，</b>
1. 完整性：单条数据是否存在空值，统计的字段是否完善。
2. 全面性：观察某一列的全部数值，比如在 Excel 表中，我们选中一列，可以看到该列的平均值、最大值、最小值。我们可以通过常识来判断该列是否有问题，比如：数据定义、单位标识、数值本身。
3. 合法性：数据的类型、内容、大小的合法性。比如数据中存在非 ASCII 字符，性别存在了未知，年龄超过了 150 岁等。
4. 唯一性：数据是否存在重复记录，因为数据通常来自不同渠道的汇总，重复的情况是常见的。行数据、列数据都需要是唯一的，比如一个人不能重复记录多次，且一个人的体重也不能在列指标中重复记录多次

### 依照“完全合一”的准则，使用 Pandas 来进行清洗。
#### 1.完整性
<b>问题 1：缺失值</b>
在数据中有些年龄、体重数值是缺失的，这往往是因为数据量较大，在过程中，有些数值没有采集到。通常我们可以采用以下三种方法：
* 删除：删除数据缺失的记录；
* 均值：使用当前列的均值；
* 高频：使用当前列出现频率最高的数据。
比如我们想对 df[‘Age’]中缺失的数值用平均年龄进行填充，可以这样写：
```python
df['Age'].fillna(df['Age'].mean(), inplace=True)
```

<b>问题 2：空行</b>
我们发现数据中有一个空行，除了 index 之外，全部的值都是 NaN。Pandas 的 read_csv() 并没有可选参数来忽略空行，这样，我们就需要在数据被读入之后再使用 dropna() 进行处理，删除空行。
```python
# 删除全空的行
df.dropna(how='all',inplace=True) 
```

#### 2.全面性
<b>问题：列数据的单位不统一</b>
观察 weight 列的数值，我们能发现 weight 列的单位不统一。有的单位是千克（kgs），有的单位是磅（lbs）。
这里我使用千克作为统一的度量单位，将磅（lbs）转化为千克（kgs）：
```python
# 获取 weight 数据列中单位为 lbs 的数据，且不包含空值
rows_with_lbs = df['weight'].str.contains('lbs').fillna(False)
print(df[rows_with_lbs])
# 将 lbs转换为 kgs, 2.2lbs=1kgs
for i,lbs_row in df[rows_with_lbs].iterrows():
  # 截取从头开始到倒数第三个字符之前，即去掉lbs。
  weight = int(float(lbs_row['weight'][:-3])/2.2)
  df.at[i,'weight'] = '{}kgs'.format(weight) 
```

#### 3.合理性
<b>问题：非 ASCII 字符</b>
我们可以看到在数据集中 Firstname 和 Lastname 有一些非 ASCII 的字符。我们可以采用删除或者替换的方式来解决非 ASCII 问题，这里我们使用删除方法：
* [x00-x7f] 匹配baiASCII值从0-127的字符，[^]表示這些字符以外的值。
```python
# 删除非 ASCII 字符
df['first_name'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
df['last_name'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
```

#### 4.唯一性
<b>问题 1：一列有多个参数</b>
在数据中不难发现，姓名列（Name）包含了两个参数 Firstname 和 Lastname。为了达到数据整洁目的，我们将 Name 列拆分成 Firstname 和 Lastname 两个字段。我们使用 Python 的 split 方法，str.split(expand=True)，将列表拆成新的列，再将原来的 Name 列删除。
```python
# 切分名字，删除源数据列
df[['first_name','last_name']] = df['name'].str.split(expand=True)
df.drop('name', axis=1, inplace=True)
```

<b>问题 2：重复数据</b>
我们校验一下数据中是否存在重复记录。如果存在重复记录，就使用 Pandas 提供的 drop_duplicates() 来删除重复数据。
```python
# 删除重复数据行
df.drop_duplicates(['first_name','last_name'],inplace=True)
```
<img src='./images/11-02.jpg'>
学习完今天的内容后，给你留个小作业吧。下面是一个美食数据，如果你拿到下面的数据，按照我们今天讲的准则，你能找到几点问题？如果你来清洗这些数据，你打算怎样清洗呢？
<img src='./images/11-02.png'>


In [1]:
data = {'food':['bacon','pulled pork','bacon','Pastrami','corned beef', 'Bacon','pastrami','honey ham','nova lox'],
       'ounces':[4.0,3.0,None,6.0,7.5,8.0,-3.0,5.0,6.0],
       'animal':['pig','pig','pig','cow','cow','pig','cow','pig','salmon']}
"""
1. 完整性: 有一格缺失值
2. 全面性: 單位不統一，food欄位值有大小寫，不一致
3. 合理性: 若這是物料控管清單，負值可能是出貨，所以減少庫存。需要跟業主確認。
4. 唯一性: 若是出貨紀錄，就沒必要要求food欄位是唯一。
"""

In [2]:
import pandas as pd
from pandas import Series, DataFrame
df = DataFrame(pd.read_excel('accountMessage.xlsx'))
df

,\t,0,1,2,3,4,5,6,7,8,9
0,0,1.0,Mickey Mouse,56.0,70kgs,72,69,71,-,-,-
1,1,2.0,Donald Dunck,34.0,154.89lbs,-,-,-,85,84,76
2,2,3.0,Mini Mouse,16.0,NaN,-,-,-,65,69,72
3,3,4.0,Scrooge McDuck,NaN,78kgs,78,79,72,-,-,-
4,4,5.0,Pink Panther,54.0,198.658lbs,-,-,-,69,NaN,75
5,5,6.0,Huey McDuck,52.0,189lbs,-,-,-,68,75,72
6,6,7.0,Dewey McDuck,19.0,56kgs,-,-,-,71,78,75
7,7,8.0,Scoopy Doo,32.0,78kgs,78,76,75,-,-,-
8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,9.0,Huey McDuck,52.0,189lbs,-,-,-,68,75,72


In [7]:
df1 = df.drop(columns=['\t',0])
df1

,1,2,3,4,5,6,7,8,9
0,Mickey Mouse,56.0,70kgs,72,69,71,-,-,-
1,Donald Dunck,34.0,154.89lbs,-,-,-,85,84,76
2,Mini Mouse,16.0,NaN,-,-,-,65,69,72
3,Scrooge McDuck,NaN,78kgs,78,79,72,-,-,-
4,Pink Panther,54.0,198.658lbs,-,-,-,69,NaN,75
5,Huey McDuck,52.0,189lbs,-,-,-,68,75,72
6,Dewey McDuck,19.0,56kgs,-,-,-,71,78,75
7,Scoopy Doo,32.0,78kgs,78,76,75,-,-,-
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Huey McDuck,52.0,189lbs,-,-,-,68,75,72


In [8]:
df1.dropna(how='all',inplace=True)
df1

In [21]:
lst = [None,'name','Age','Weight','m0006','m0612','m1218','f0006','f0612','f1218']
df2 = df1.rename(columns=dict(enumerate(lst)),inplace=False)
df2

,name,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218
0,Mickey Mouse,56.0,70kgs,72,69,71,-,-,-
1,Donald Dunck,34.0,154.89lbs,-,-,-,85,84,76
2,Mini Mouse,16.0,NaN,-,-,-,65,69,72
3,Scrooge McDuck,NaN,78kgs,78,79,72,-,-,-
4,Pink Panther,54.0,198.658lbs,-,-,-,69,NaN,75
5,Huey McDuck,52.0,189lbs,-,-,-,68,75,72
6,Dewey McDuck,19.0,56kgs,-,-,-,71,78,75
7,Scoopy Doo,32.0,78kgs,78,76,75,-,-,-
9,Huey McDuck,52.0,189lbs,-,-,-,68,75,72
10,Louie McDuck,12.0,45kgs,-,-,-,92,95,87


In [22]:
df2['Age'].fillna(df2['Age'].mean(), inplace=True)
df2

In [24]:
rows_with_lbs = df2['Weight'].str.contains('lbs')

In [25]:
rows_with_lbs

0     False
1      True
2       NaN
3     False
4      True
5      True
6     False
7     False
9      True
10    False
Name: Weight, dtype: object

In [26]:
rows_with_lbs = df2['Weight'].str.contains('lbs').fillna(False)  #若遇到空值，為False

In [27]:
rows_with_lbs

0     False
1      True
2     False
3     False
4      True
5      True
6     False
7     False
9      True
10    False
Name: Weight, dtype: bool

In [28]:
type(rows_with_lbs)

pandas.core.series.Series

In [29]:
df2[rows_with_lbs]  #列出為True的幾筆

,name,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218
1,Donald Dunck,34.0,154.89lbs,-,-,-,85,84,76
4,Pink Panther,54.0,198.658lbs,-,-,-,69,NaN,75
5,Huey McDuck,52.0,189lbs,-,-,-,68,75,72
9,Huey McDuck,52.0,189lbs,-,-,-,68,75,72


In [47]:
# itterows()迭代器，每筆資料皆為tuple格式
# i為該筆資料的index，lbs_weight為該筆所有資料，帶入colname，可以取出該值
"""
name      Donald Dunck
Age               34.0
Weight       154.89lbs
m0006                -
m0612                -
m1218                -
f0006               85
f0612               84
f1218               76
"""
for i,lbs_weight in df2[rows_with_lbs].iterrows():
    weight = int(float(lbs_weight['Weight'][:-3])/2.2)
    df2.at[i, 'Weight'] = f'{weight}kgs'
    

In [50]:
df2.at[3,'Age'] = int(df2.at[3,'Age'])

In [51]:
df2

,name,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218
0,Mickey Mouse,56.0,70kgs,72,69,71,-,-,-
1,Donald Dunck,34.0,70kgs,-,-,-,85,84,76
2,Mini Mouse,16.0,NaN,-,-,-,65,69,72
3,Scrooge McDuck,36.0,78kgs,78,79,72,-,-,-
4,Pink Panther,54.0,90kgs,-,-,-,69,NaN,75
5,Huey McDuck,52.0,85kgs,-,-,-,68,75,72
6,Dewey McDuck,19.0,56kgs,-,-,-,71,78,75
7,Scoopy Doo,32.0,78kgs,78,76,75,-,-,-
9,Huey McDuck,52.0,85kgs,-,-,-,68,75,72
10,Louie McDuck,12.0,45kgs,-,-,-,92,95,87


In [54]:
df2[['first_name','last_name']] = df2['name'].str.split(expand=True)

In [56]:
df2.drop('name', axis=1,inplace=True)

In [57]:
df2

,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,first_name,last_name
0,56.0,70kgs,72,69,71,-,-,-,Mickey,Mouse
1,34.0,70kgs,-,-,-,85,84,76,Donald,Dunck
2,16.0,NaN,-,-,-,65,69,72,Mini,Mouse
3,36.0,78kgs,78,79,72,-,-,-,Scrooge,McDuck
4,54.0,90kgs,-,-,-,69,NaN,75,Pink,Panther
5,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
6,19.0,56kgs,-,-,-,71,78,75,Dewey,McDuck
7,32.0,78kgs,78,76,75,-,-,-,Scoopy,Doo
9,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
10,12.0,45kgs,-,-,-,92,95,87,Louie,McDuck


In [59]:
list(df2.columns)

['Age',
 'Weight',
 'm0006',
 'm0612',
 'm1218',
 'f0006',
 'f0612',
 'f1218',
 'first_name',
 'last_name']

In [60]:
df2 = df2.reindex(columns=['first_name',
 'last_name',
 'Age',
 'Weight',
 'm0006',
 'm0612',
 'm1218',
 'f0006',
 'f0612',
 'f1218',
])

In [65]:
df2

,first_name,last_name,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218
0,Mickey,Mouse,56.0,70kgs,72,69,71,-,-,-
1,Donald,Dunck,34.0,70kgs,-,-,-,85,84,76
2,Mini,Mouse,16.0,NaN,-,-,-,65,69,72
3,Scrooge,McDuck,36.0,78kgs,78,79,72,-,-,-
4,Pink,Panther,54.0,90kgs,-,-,-,69,NaN,75
5,Huey,McDuck,52.0,85kgs,-,-,-,68,75,72
6,Dewey,McDuck,19.0,56kgs,-,-,-,71,78,75
7,Scoopy,Doo,32.0,78kgs,78,76,75,-,-,-
9,Huey,McDuck,52.0,85kgs,-,-,-,68,75,72
10,Louie,McDuck,12.0,45kgs,-,-,-,92,95,87
